# Data Cleaning

Cleaning and parsing of data from the [NBCUniversal Analytics Challenge](http://sc.aisnet.org/conference2018/student-competitions/nbcuniversal-challenge/).

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Loading Data

In [2]:
data = pd.read_csv('Data/NBCU-dataLaurel.csv')
data.head()

,imdbid,title,plot,rating,imdb_rating,metacritic,dvd_release,production,actors,imdb_votes,poster,director,release_date,runtime,genre,awards,keywords,Budget,Box Office Gross
0,tt0010323,The Cabinet of Dr. Caligari,"Hypnotist Dr. Caligari uses a somnambulist, Ce...",UNRATED,8.1,NaN,15-Oct-97,Rialto Pictures,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...","42,583",https://images-na.ssl-images-amazon.com/images...,Robert Wiene,19-Mar-21,67 min,"Fantasy, Horror, Mystery",1 nomination.,expressionism|somnambulist|avant-garde|hypnosi...,18000,0
1,tt0052893,Hiroshima Mon Amour,A French actress filming an anti-war film in H...,NOT RATED,8.0,NaN,24-Jun-03,Rialto Pictures,"Emmanuelle Riva, Eiji Okada, Stella Dassas, Pi...","21,154",https://images-na.ssl-images-amazon.com/images...,Alain Resnais,16-May-60,90 min,"Drama, Romance",Nominated for 1 Oscar. Another 6 wins & 5 nomi...,memory|atomic-bomb|lovers-separation|impossibl...,88300,0
2,tt0058898,Alphaville,A U.S. secret agent is sent to the distant spa...,NOT RATED,7.2,NaN,20-Oct-98,Rialto Pictures,"Eddie Constantine, Anna Karina, Akim Tamiroff","17,801",https://images-na.ssl-images-amazon.com/images...,Jean-Luc Godard,5-May-65,99 min,"Drama, Mystery, Sci-Fi",1 win.,dystopia|french-new-wave|satire|comic-violence...,220000,46585
3,tt0074252,"Ugly, Dirty and Bad",Four generations of a family live crowded toge...,NaN,7.9,NaN,1-Nov-16,Compagnia Cinematografica Champion,"Nino Manfredi, Maria Luisa Santella, Francesco...","5,705",https://images-na.ssl-images-amazon.com/images...,Ettore Scola,23-Sep-76,115 min,"Comedy, Drama",1 win & 2 nominations.,incest|failed-murder-attempt|poisoned-food|bap...,6590,0
4,tt0084269,Losing Ground,A comedy-drama about a Black American female p...,NaN,6.3,NaN,NaN,Milestone Film & Video,"Billie Allen, Gary Bolling, Clarence Branch Jr...",132,https://images-na.ssl-images-amazon.com/images...,Kathleen Collins,1-Jun-82,86 min,"Comedy, Drama",NaN,artist|painter|marriage|black-independent-film...,0,0


In [3]:
data.shape

(8468, 19)

These are the variables we have to work with:

imdbid: Unique Id used by IMDB to refer to the movie.

Title: Title of the movie

plot: Movie plot summary

rating: MPAA Appropriate audience rating

imdb_rating: IMDB's voters' scoring of a movie on a scale from 1-10 (10 being best)

metacritic: Metacritic movie score on a scale of 0-100 (100 being best)

dvd_release: Movie release date on DVD

production: Principle production company

actors: Lead Actors

imdb_votes: Total votes from IMDB members

poster: Movie Poster artwork

director: Movie director

release_date: Theatrical Release Date

runtime: Runtime length of movie in minutes

genre: Genre Classification

awards: Academy awards & nominations

keywords: Keywords associated with the movie

budget: Budget spent on movie production, marketing, and distribution

box office gross: Box Office Gross Returns as of 9/21/2017

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null object
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null object
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null object
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: float64(2), object(17)
memory usage: 1.2+ MB


Notice how many of variables are just objects. We're going to have to deal with converting a few of these into useful types.

### Parsing imdbid

The proper format for imdb id's is simply 7 digits, with no tt.

In [5]:
data['imdbid'] = data['imdbid'].str.replace('tt','')

### Parsing release_dates and dvd_release

Next, we'll change release_date to a datetime-like type.

In [6]:
data['release_date'].head()

0    19-Mar-21
1    16-May-60
2     5-May-65
3    23-Sep-76
4     1-Jun-82
Name: release_date, dtype: object

In [7]:
pd.to_datetime(data['release_date'])[1],data['release_date'][1]

(Timestamp('2060-05-16 00:00:00'), '16-May-60')

Then we see that there is an issue with pandas to_datetime function. It converts very old dates back to the 19th century. Perhaps we need to use the datetime package per [this](https://stackoverflow.com/questions/16600548/how-to-parse-string-dates-with-2-digit-year).

In [8]:
dates = data['release_date']
dates = pd.to_datetime(dates)
opivot_index = dates[dates.apply(lambda x: x.year>2019)].index
for index in opivot_index:
    dates[index] = dates[index].replace(year = dates[index].year-100)
sum(dates>'2019-12-31')

0

Then we've found a way to account for Python's default pivot year.

In [9]:
data['release_date']=dates
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null object
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null object
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null datetime64[ns]
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: datetime64[ns](1), float64(2), object(16)
memory usage: 1.2+ MB


It seems natural to also do the same for dvd_release.

In [10]:
dates = data['dvd_release']
dates = pd.to_datetime(dates)
opivot_index = dates[dates.apply(lambda x: x.year>2019)].index
for index in opivot_index:
    dates[index] = dates[index].replace(year = dates[index].year-100)
data['dvd_release'] = dates
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null datetime64[ns]
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null object
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null datetime64[ns]
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: datetime64[ns](2), float64(2), object(15)
memory usage: 1.2+ MB


### Parsing imdb_votes

Next, we have several important numerical variables that are currenly in object types. First, we'll work with imdb_votes.

In [11]:
data['imdb_votes'].head()

0    42,583
1    21,154
2    17,801
3     5,705
4       132
Name: imdb_votes, dtype: object

So we need to convert imdb_votes to integers. Since there are commas in each number, we cannot simply tell pandas to treat each entry as an integer via the .astype() function. We'll first have to replace each comma with a blank, then apply the int() function. We also have to take care to ignore all of the missing values from imdb_votes as we will be dealing with those later.

In [12]:
votes = data['imdb_votes']
votes_parse = votes.str.replace(',','')
votes_parse.head()

0    42583
1    21154
2    17801
3     5705
4      132
Name: imdb_votes, dtype: object

Then, we've successfully removed all of the commas. Let's confirm that we didn't lose any datapoints along the way.

In [13]:
[len(votes_parse), len(data['imdb_votes'])]

[8468, 8468]

In order to convert to int, we have to find a way to work around missing values. Let's replace all the missing values with -1 and then convert them back to NaN after conversion.

In [14]:
import numpy as np

votes_int = votes_parse.fillna(-1).astype('int')
votes_int[votes_int==-1] = np.nan
votes_int.isna().sum() == votes_parse.isna().sum()

True

In [15]:
data['imdb_votes'] = votes_int
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8468 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8468 non-null object
title               8468 non-null object
plot                8196 non-null object
rating              5252 non-null object
imdb_rating         7735 non-null float64
metacritic          5079 non-null float64
dvd_release         5335 non-null datetime64[ns]
production          6758 non-null object
actors              8153 non-null object
imdb_votes          7735 non-null float64
poster              7967 non-null object
director            8390 non-null object
release_date        8283 non-null datetime64[ns]
runtime             7846 non-null object
genre               8424 non-null object
awards              5242 non-null object
keywords            6381 non-null object
Budget              8468 non-null object
Box Office Gross    8468 non-null object
dtypes: datetime64[ns](2), float64(3), object(14)
memory usage: 1.2+ MB


### Parsing Budget and Box Office Gross

Now we have to deal with 'Budget' and 'Box Office Gross' in a similar manner.

In [16]:
data[['Budget', 'Box Office Gross']].head()

,Budget,Box Office Gross
0,18000,0
1,88300,0
2,220000,46585
3,6590,0
4,0,0


Looks like we don't have to worry about any commas in 'Budget' or in 'Box Office Gross', so the conversions will be much simplier. In the previous revision of Data Cleaning, we found that data entries containing Box Office Gross values in foreign currencies were not worth saving. We will be removing them in this revision.

In [17]:
data = data.drop(data[data['Box Office Gross'].str.contains('GBP')].index)
data = data.drop(data[data['Box Office Gross'].str.contains('EU')].index)
data = data.drop(data[data['Budget'].str.contains('EU')].index)
data = data.drop(data[data['Budget'].str.contains('CAD')].index)

Now that we've removed all datapoints with foreign currencies, we can start parsing Budget and Box Office Gross into numerical types.

In [18]:
gross = data['Box Office Gross']
gross = gross.str.replace(',','')

error_index = []

for i, item in enumerate(gross):
    try:
        int(item)
    except ValueError:
        error_index.append(i)

In [19]:
gross[error_index]

/usr/local/lib/python3.7/site-packages/pandas/core/series.py:942: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


1394                                                    0
1869                                                    0
2914                                                    0
3866                                                    0
5093                                             14793904
5684                                                    0
5723                                                    0
5828                                                18202
5981    <strongitemprop="name">AGrosslyunderratedmovie...
6277                                                  NaN
6902                                                    0
7075                                                    0
7137                                                    0
Name: Box Office Gross, dtype: object

This is peculiar, the value of Box Office Gross seems to change once we store that variable as gross (from these string entries to the number 0). On top of that, the original value in the dataset seems to be random text, perhaps a misscraped comment?

**Edit**: On review, enumerate gives us an enumerated index, so to find the erroneous entries, we need to use df.iloc

When I review the data entries for 1394, it's hard to see what relevance '<strongitemprop="name">Grossbuttotallyworthwhile</strong>' has. Then, we'll remove these samples from the dataset.

In [20]:
error_index = data.iloc[error_index].index
data = data.drop(error_index)

Then, we can finally change box office gross and budget into integer formats.

In [21]:
data['Box Office Gross'] = data['Box Office Gross'].astype('int')
data['Budget'] = data['Budget'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8206 entries, 0 to 8467
Data columns (total 19 columns):
imdbid              8206 non-null object
title               8206 non-null object
plot                7951 non-null object
rating              5092 non-null object
imdb_rating         7510 non-null float64
metacritic          4904 non-null float64
dvd_release         5172 non-null datetime64[ns]
production          6568 non-null object
actors              7904 non-null object
imdb_votes          7510 non-null float64
poster              7729 non-null object
director            8131 non-null object
release_date        8028 non-null datetime64[ns]
runtime             7607 non-null object
genre               8163 non-null object
awards              5065 non-null object
keywords            6177 non-null object
Budget              8206 non-null int64
Box Office Gross    8206 non-null int64
dtypes: datetime64[ns](2), float64(3), int64(2), object(12)
memory usage: 1.3+ MB


### Parsing imdb_rating

Since imdb_rating has many unique values, we're going to round all the ratings down to the whole number. We can use train on the ranks themselves in decision trees, but we'll have to resort to one-hot encoding for regression, neural nets, etc.

In [22]:
data['imdb_rating'].isna().sum()

696

There are 696 data points missing imdb ratings. We'll try using imdb packages like [imdbpy](https://imdbpy.readthedocs.io/en/latest/) to fill in these values.

**Edit**: Turns out, imdbpy is only useful for getting the release *year* of a movie, not the release *date*. For now, it seems like all we can do is remove these data points.

In [23]:
na_index = data[data['imdb_rating'].isna()].index
data = data.drop(na_index)

In [24]:
import math

data['imdb_rating'] = data['imdb_rating'].apply(lambda x: math.floor(x))

### Parsing runtime

For runtime, we would like to remove the 'min' part of each entry. It's much better for analysis to just assume that runtime is measured in minutes.

In [25]:
data['runtime'].isna().sum()

142

In [26]:
from imdb import IMDb
ia = IMDb()

Unlike previously, imdbpy actualy has a feature to get movie runtimes from imdb. Because of this, we will be able to fill in the missing values.

In [27]:
na_index = data[data['runtime'].isna()].index
for index in na_index:
    movie = ia.get_movie(data.loc[index]['imdbid'])
    try:
        data.loc[index]['imdbid'] = movie['runtime']
    except KeyError:
        data = data.drop(index)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [28]:
data[data['runtime'].isna()]

,imdbid,title,plot,rating,imdb_rating,metacritic,dvd_release,production,actors,imdb_votes,poster,director,release_date,runtime,genre,awards,keywords,Budget,Box Office Gross
832,1935277,Road to Juarez,An American ex-con with Mexican underworld tie...,NaN,5,NaN,NaT,Mousetrap Films,"William Forsythe, Jacqueline Pinol, Pepe Serna...",43.0,http://ia.media-imdb.com/images/M/MV5BNjM3MjUw...,David A. Ponce de Leon,2015-04-24,NaN,"Action, Thriller",NaN,NaN,0,0
1989,3305388,Mountain Top,NaN,NaN,8,NaN,NaT,NaN,"Barry Corbin, Coby Ryan McLaughlin, Valerie Az...",46.0,NaN,Gary Wheeler,2014-05-05,NaN,"Drama, Family, Mystery",NaN,NaN,750000,0
2267,3640942,Koyelaanchal,Koyelaanchal (coal belt of India) brings to li...,NaN,5,NaN,NaT,NaN,"Kannan Arunachalam, Biswanath Basu, Vinod Khan...",171.0,https://images-na.ssl-images-amazon.com/images...,Ashu Trikha,2014-05-09,NaN,"Action, Drama",1 nomination.,NaN,0,0
2428,3822606,Ra Ra Krishnayya,Kittu alias Krishna Sundeep Kishan is a cab dr...,NaN,5,NaN,NaT,NaN,"Jagapathi Babu, Ravi Babu, Tanikella Bharani, ...",82.0,https://images-na.ssl-images-amazon.com/images...,Mahesh P.,2014-07-04,NaN,Romance,NaN,NaN,0,0
2530,3969208,Trust Fund,"Reese Donahue leads a seemingly ideal life, wi...",PG,7,NaN,NaT,Transatlantic Films,"Matthew Alan, Jessica Rothe, Willie Garson, An...",22.0,https://images-na.ssl-images-amazon.com/images...,Sandra L. Martin,2016-01-08,NaN,Drama,NaN,NaN,0,0
2588,4074296,From This Day Forward,When director Sharon Shattuck's father came ou...,NaN,7,NaN,NaT,Argot Pictures,NaN,12.0,http://ia.media-imdb.com/images/M/MV5BMjAxNzkx...,Sharon Shattuck,2015-04-11,NaN,"Documentary, Biography, Family",NaN,f-rated,0,0
2840,4489160,You're My Boss,"A woman who is looking for acceptance, who's l...",NaN,6,NaN,NaT,NaN,"Toni Gonzaga, Coco Martin, Freddie Webb, JM de...",102.0,https://images-na.ssl-images-amazon.com/images...,Antoinette Jadaone,2015-04-04,NaN,"Comedy, Romance",2 wins & 11 nominations.,personal-assistant|acceptance|runner|boss|company,0,0
2900,4621100,Nanak Shah Fakir,Nanak Shah Fakir is a biographical film on the...,NaN,9,NaN,NaT,B4U US,"Arif Zakaria, Puneet Sikka, Adil Hussain, Anur...",107.0,https://images-na.ssl-images-amazon.com/images...,NaN,2015-04-17,NaN,Drama,3 wins.,NaN,0,0
2914,4641602,365 Days,NaN,NaN,7,NaN,NaT,NaN,"Anand, Anaika Soti",17.0,http://ia.media-imdb.com/images/M/MV5BYzlkZTg3...,Ram Gopal Varma,2015-05-22,NaN,Drama,NaN,written-by-director|number-in-title,0,0
3069,4944460,Everyday I Love You,Two people bound together in the same journey ...,NaN,7,NaN,NaT,Star Cinema,"Gerald Anderson, Enrique Gil, Liza Soberano",195.0,https://images-na.ssl-images-amazon.com/images...,Mae Czarina Cruz,2015-11-06,NaN,"Drama, Romance",2 nominations.,NaN,0,0


In [29]:
na_index = data[data['runtime'].isna()].index
runtimes = []
for index in na_index:
    movie = ia.get_movie(data.loc[index]['imdbid'])
    try:
        runtimes.append(movie['runtime'][0])
    except KeyError:
        data = data.drop(index)

In [30]:
data['runtime'].isna().sum()

72

In [31]:
na_data = data[data['runtime'].isna()]

In [32]:
for index in na_data.head().index:
    movie = ia.get_movie(na_data.loc[index]['imdbid'])
    na_data.loc[index]['runtime'] = movie['runtime']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
na_data.head().index

Int64Index([832, 1989, 2267, 2428, 2530], dtype='int64')

In [34]:
movie = ia.get_movie('1935277')
na_data.loc[832]['runtime'] = movie['runtime']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [35]:
runtimes1 = na_data['imdbid'].apply(lambda x: ia.get_movie(x)['runtime'][0])

In [36]:
na_data['runtime'] = na_data['runtime'].fillna(runtimes1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
data['runtime'] = data['runtime'].fillna(runtimes1)

In [38]:
data['runtime'].isna().sum()

0

Now that we've filled in all of the missing runtimes that we could possibly fill in, we can drop ' min' and convert to integer type.

In [39]:
data['runtime'] = data['runtime'].str.replace(' min', '')
data['runtime'] = data['runtime'].astype('int')

### Parsing genre

Now let's parse through genre in order to use it as a categorical variable.

In [40]:
genres = data['genre']

Thankfully, imdbpy has a function to retrieve genres for movies when available. We're going to attempt to fill in these values and drop the ones in which we can't.

In [41]:
na_genres = genres[genres.isna()].index

In [42]:
genres = []
for index in na_genres:
    movie = ia.get_movie(data.loc[index]['imdbid'])
    try:
        genres.append(movie['genres'][0])
    except KeyError:
        data = data.drop(index)

In [43]:
na_data = data[data['genre'].isna()]
genres1 = na_data['imdbid'].apply(lambda x: ia.get_movie(x)['genre'][0])

In [44]:
data['genre'] = data['genre'].fillna(genres1)

In [45]:
genre_index = data['genre'].index

In [46]:
for i in genre_index:
    parsed_genre = [x.strip() for x in data.loc[i]['genre'].split(',')]

In [47]:
genres = data['genre']

In [48]:
parsed_genres = genres.apply(lambda x: [y.strip() for y in x.split(',')])

In [49]:
genres = pd.Series(parsed_genres.sum()).unique()

Now that we've separated all of the distinct genres out, we have to create categorical variables for each.

In [50]:
for genre in genres:
    print(genre)

Fantasy
Horror
Mystery
Drama
Romance
Sci-Fi
Comedy
Crime
Action
Adventure
Animation
Family
History
Thriller
Biography
Documentary
Music
Sport
Western
Musical
War
News
Short


In [81]:
for i in range(len(parsed_genres[0])):
    print(parsed_genres[0][i])

Fantasy
Horror
Mystery


In [85]:
test = pd.DataFrame([genres])

In [91]:
data.shape[0]

7439

Now that we've separated out all of the genres in 'genre', we can create categorical variables for each category. Then, we can apply one-hot encoding to encode movie genres as categorical variables.

In [95]:
genre_classes = pd.DataFrame(0, index = data.index, columns = genres)

In [100]:
parsed_genres[0][i]

'Mystery'

In [108]:
index = 8464

In [112]:
genre_classes.loc[index][parsed_genres[index]] = 1

In [115]:
for index in data.index:
    genre_classes.loc[index][parsed_genres[index]] = 1

In [133]:
test = pd.merge(data, genre_classes, left_index=True, right_index=True)

In [140]:
test[genres].astype('category')

,Fantasy,Horror,Mystery,Drama,Romance,Sci-Fi,Comedy,Crime,Action,Adventure,...,Thriller,Biography,Documentary,Music,Sport,Western,Musical,War,News,Short
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
